In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pytz
import ta

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
from datetime import datetime
import MetaTrader5 as mt5

from ortisan_ta.dataaccess import DataItem, MetaTraderDataAccess
import ortisan_ta.utils.analysis as ortisan_ta

from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volume import OnBalanceVolumeIndicator
from ortisan_ta.simulator import MarketSimulator


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#Settings
mpl.style.use('seaborn')
default_fig_size = (20, 20) # Width , Height

### Symbols in my historical database

In [59]:
symbols = ["ABEV3",
"AZUL4",
"B3SA3",
"BBAS3",
"BBDC4",
"BBSE3",
"BPAC11",
"BRDT3",
"BRFS3",
"BRML3",
"BTOW3",
"CCRO3",
"CIEL3",
"CMIG4",
"COGN3",
"CSNA3",
"CVCB3",
"CYRE3",
"ELET3",
"EQTL3",
"GGBR4",
"GNDI3",
"GOAU4",
"GOLL4",
"IRBR3",
"ITSA4",
"ITUB4",
"JBSS3",
"KLBN11",
"LAME4",
"LREN3",
"MGLU3",
"MRFG3",
"MULT3",
"NTCO3",
"PETR3",
"PETR4",
"PRIO3",
"RADL3",
"RAIL3",
"RENT3",
"SBSP3",
"SULA11",
"SUZB3",
"TOTS3",
"UGPA3",
"USIM5",
"VALE3",
"VVAR3",
"WEGE3"]


In [60]:
symbols

['ABEV3',
 'AZUL4',
 'B3SA3',
 'BBAS3',
 'BBDC4',
 'BBSE3',
 'BPAC11',
 'BRDT3',
 'BRFS3',
 'BRML3',
 'BTOW3',
 'CCRO3',
 'CIEL3',
 'CMIG4',
 'COGN3',
 'CSNA3',
 'CVCB3',
 'CYRE3',
 'ELET3',
 'EQTL3',
 'GGBR4',
 'GNDI3',
 'GOAU4',
 'GOLL4',
 'IRBR3',
 'ITSA4',
 'ITUB4',
 'JBSS3',
 'KLBN11',
 'LAME4',
 'LREN3',
 'MGLU3',
 'MRFG3',
 'MULT3',
 'NTCO3',
 'PETR3',
 'PETR4',
 'PRIO3',
 'RADL3',
 'RAIL3',
 'RENT3',
 'SBSP3',
 'SULA11',
 'SUZB3',
 'TOTS3',
 'UGPA3',
 'USIM5',
 'VALE3',
 'VVAR3',
 'WEGE3']

### Gets the data for each symbol

In [89]:
data_access = MetaTraderDataAccess()
dfs = data_access.get_rates_from_symbols(symbols, datetime(2020, 1, 1), datetime(2020, 12, 4), mt5.TIMEFRAME_D1)
dfs

{'ABEV3':                       Open   High    Low  Close  Spread    Volume
 Date                                                             
 2020-01-01 21:00:00  18.45  18.83  18.37  18.78       1  16411300
 2020-01-02 21:00:00  18.54  18.74  18.48  18.52       1  20059000
 2020-01-05 21:00:00  18.45  18.75  18.28  18.60       1  14644300
 2020-01-06 21:00:00  18.63  18.81  18.45  18.64       1  18036600
 2020-01-07 21:00:00  18.67  18.74  18.45  18.55       1  14051700
 ...                    ...    ...    ...    ...     ...       ...
 2020-11-29 21:00:00  14.10  14.18  13.67  13.67       1  41880800
 2020-11-30 21:00:00  13.78  14.42  13.77  14.36       1  26702100
 2020-12-01 21:00:00  14.29  14.60  14.16  14.55       1  18287100
 2020-12-02 21:00:00  14.42  14.53  13.99  14.38       1  32918200
 2020-12-03 21:00:00  14.42  14.49  14.24  14.42       1  18361200
 
 [233 rows x 6 columns],
 'AZUL4':                       Open   High    Low  Close  Spread    Volume
 Date            

### Filtering dfs with data (df is not empty)

In [90]:
dfs_with_data = dict(filter(lambda elem: not elem[1].empty, dfs.items()))

In [91]:
dfs_with_data

{'ABEV3':                       Open   High    Low  Close  Spread    Volume
 Date                                                             
 2020-01-01 21:00:00  18.45  18.83  18.37  18.78       1  16411300
 2020-01-02 21:00:00  18.54  18.74  18.48  18.52       1  20059000
 2020-01-05 21:00:00  18.45  18.75  18.28  18.60       1  14644300
 2020-01-06 21:00:00  18.63  18.81  18.45  18.64       1  18036600
 2020-01-07 21:00:00  18.67  18.74  18.45  18.55       1  14051700
 ...                    ...    ...    ...    ...     ...       ...
 2020-11-29 21:00:00  14.10  14.18  13.67  13.67       1  41880800
 2020-11-30 21:00:00  13.78  14.42  13.77  14.36       1  26702100
 2020-12-01 21:00:00  14.29  14.60  14.16  14.55       1  18287100
 2020-12-02 21:00:00  14.42  14.53  13.99  14.38       1  32918200
 2020-12-03 21:00:00  14.42  14.49  14.24  14.42       1  18361200
 
 [233 rows x 6 columns],
 'AZUL4':                       Open   High    Low  Close  Spread    Volume
 Date            

### Pivot Points

In [92]:
example_asset = "ITUB4"
df = dfs_with_data[example_asset]
df



,Open,High,Low,Close,Spread,Volume
Date,,,,,,
2020-01-01 21:00:00,35.88,36.60,35.60,36.60,1,20666100
2020-01-02 21:00:00,36.09,36.80,36.04,36.21,1,24891400
2020-01-05 21:00:00,36.14,36.16,35.52,35.67,1,22294700
2020-01-06 21:00:00,35.67,35.84,34.85,34.85,1,20000900
2020-01-07 21:00:00,35.08,35.42,34.28,34.28,1,25980900
...,...,...,...,...,...,...
2020-11-29 21:00:00,28.89,29.12,28.48,28.48,1,59889300
2020-11-30 21:00:00,28.99,30.40,28.88,29.68,1,51364700
2020-12-01 21:00:00,29.85,30.17,29.38,29.91,1,32159500


In [116]:
df_shifted = df.shift(1)
df_shifted.head(10)

,Open,High,Low,Close,Spread,Volume
Date,,,,,,
2020-01-01 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02 21:00:00,35.88,36.60,35.60,36.60,1.0,20666100.0
2020-01-05 21:00:00,36.09,36.80,36.04,36.21,1.0,24891400.0
2020-01-06 21:00:00,36.14,36.16,35.52,35.67,1.0,22294700.0
2020-01-07 21:00:00,35.67,35.84,34.85,34.85,1.0,20000900.0
2020-01-08 21:00:00,35.08,35.42,34.28,34.28,1.0,25980900.0
2020-01-09 21:00:00,34.44,34.58,33.22,33.60,1.0,48423800.0
2020-01-12 21:00:00,33.76,34.12,33.27,33.30,1.0,25521600.0
2020-01-13 21:00:00,33.70,34.07,33.43,33.74,1.0,29200700.0


In [117]:
pivot = (df_shifted["High"] + df_shifted["Low"] + df_shifted["Close"]) / 3
r1 = 2 * pivot - df_shifted["Low"]
r2 = pivot + df_shifted["High"] - df_shifted["Low"]
r3 = r1 + df_shifted["High"] - df_shifted["Low"]

s1 = 2 * pivot - df_shifted["High"]
s2 = pivot - df_shifted["High"] - df_shifted["Low"]
s3 = s1 - df_shifted["High"] - df_shifted["Low"]

pivot.head(10)

Date
2020-01-01 21:00:00          NaN
2020-01-02 21:00:00    36.266667
2020-01-05 21:00:00    36.350000
2020-01-06 21:00:00    35.783333
2020-01-07 21:00:00    35.180000
2020-01-08 21:00:00    34.660000
2020-01-09 21:00:00    33.800000
2020-01-12 21:00:00    33.563333
2020-01-13 21:00:00    33.746667
2020-01-14 21:00:00    33.603333
dtype: float64

In [121]:
up_next_day = df["Close"]  > r1
down_next_day = df["Close"] < s1

close_up = df[up_next_day][DataItem.CLOSE]
close_down = df[down_next_day][DataItem.CLOSE]

df['pivot_signal'] = "0"


close_up

Date
2020-01-22 21:00:00    33.20
2020-02-06 21:00:00    32.46
2020-02-09 21:00:00    33.01
2020-02-10 21:00:00    33.77
2020-02-18 21:00:00    32.85
2020-03-09 21:00:00    28.21
2020-03-12 21:00:00    25.96
2020-03-23 21:00:00    22.28
2020-03-24 21:00:00    23.37
2020-03-29 21:00:00    23.88
2020-04-05 21:00:00    21.98
2020-04-06 21:00:00    22.79
2020-04-07 21:00:00    23.96
2020-04-27 21:00:00    23.14
2020-05-04 21:00:00    22.51
2020-05-07 21:00:00    22.13
2020-05-13 21:00:00    22.36
2020-05-17 21:00:00    22.48
2020-05-20 21:00:00    22.65
2020-05-24 21:00:00    23.76
2020-05-31 21:00:00    23.48
2020-06-01 21:00:00    25.06
2020-06-02 21:00:00    25.72
2020-06-03 21:00:00    26.47
2020-06-04 21:00:00    27.05
2020-06-07 21:00:00    28.14
2020-06-15 21:00:00    25.98
2020-06-17 21:00:00    27.34
2020-06-28 21:00:00    26.28
2020-06-30 21:00:00    25.92
2020-07-01 21:00:00    26.36
2020-07-05 21:00:00    27.53
2020-07-20 21:00:00    27.50
2020-07-26 21:00:00    27.60
2020-07-2

In [123]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df[DataItem.CLOSE], name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=close_down.index,
    y=close_down, name='Gap down', mode='markers'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=close_up.index,
    y=close_up, name='Gap up', mode='markers'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=pivot.index,
    y=pivot, name='Pivot',
), row=1, col=1)

#fig.update_layout(height=1000, width=1000, title_text="Macd Study", title_x=0.5, xaxis_tickangle=90, xaxis_tickformat = '%Y-%m-%d-%H:%M:%S')
fig.update_layout(height=1000, width=1000, title_text="PIVOT Study", title_x=0.5)
fig.show()

### Dummy buy/sell signal